In [9]:
import os, sys
import numpy as np
from collections import OrderedDict as OD
import math
#import uproot3
import uproot as uproot
import hist
import matplotlib.pyplot as plt
from matplotlib.collections import PatchCollection
from matplotlib.patches import Rectangle
import enum
import mplhep as hep
from parse import *
import copy
import json

sys.path.append( os.path.abspath('../') )
print(f"{os.path.abspath('../') = }")

from htoaa_CommonTools import (
    rebinTH1, rebinTH2, variableRebinTH1,
)

sIpFile = '/eos/cms/store/user/ssawant/htoaa/analysis/20240809_gg0l_FullSyst/2018/analyze_htoaa_stage1.root'
sOpDir0  = '/eos/cms/store/user/ssawant/htoaa/analysis/20240809_gg0l_FullSyst/2018/2DAlphabet_inputFiles'

Era = '2018' # '2018, 'Run2
#CAT = 'gg0lHi' # 'gg0l', 'VBFjj', 'Wlv', 'Zll', 'Zvv',  'Vjj'. 'ZvvIncl','ZvvLo', 'ZvvHi', 'gg0lIncl', 'gg0lLo', 'gg0lHi', 
CATAGORIES = {
    #"ttbTbjjHi": "tt0l_ge1NonHFatJet_1BExtra_Hi", 
    #"ttbTbjjMed": "tt0l_ge1NonHFatJet_1BExtra_Med", 
    #"ttbTbjjLo": "tt0l_ge1NonHFatJet_1BExtra_Lo"
    "gg0lIncl" : "gg0lIncl",
    
}


fIpFile = uproot.open(sIpFile)

os.path.abspath('../') = '/afs/cern.ch/work/s/ssawant/private/htoaa/htoaa_b_ana_SS'


In [10]:

processes_dict = {
    'Data': ['JetHT_Run2018A', 'JetHT_Run2018B', 'JetHT_Run2018C', 'JetHT_Run2018D'],

    'ggHtoaato4b_mA_12': ['ggHtoaato4b_mA_12'],
    'ggHtoaato4b_mA_15': ['ggHtoaato4b_mA_15'],
    'ggHtoaato4b_mA_20': ['ggHtoaato4b_mA_20'],
    'ggHtoaato4b_mA_25': ['ggHtoaato4b_mA_25'],
    'ggHtoaato4b_mA_30': ['ggHtoaato4b_mA_30'],
    'ggHtoaato4b_mA_35': ['ggHtoaato4b_mA_35'],
    'ggHtoaato4b_mA_40': ['ggHtoaato4b_mA_40'],
    'ggHtoaato4b_mA_45': ['ggHtoaato4b_mA_45'],
    'ggHtoaato4b_mA_50': ['ggHtoaato4b_mA_50'],
    'ggHtoaato4b_mA_55': ['ggHtoaato4b_mA_55'],
    'ggHtoaato4b_mA_60': ['ggHtoaato4b_mA_60'],

    'VBFHtoaato4b_mA_12': ['VBFHtoaato4b_mA_12'],
    'VBFHtoaato4b_mA_15': ['VBFHtoaato4b_mA_15'],
    'VBFHtoaato4b_mA_20': ['VBFHtoaato4b_mA_20'],
    'VBFHtoaato4b_mA_25': ['VBFHtoaato4b_mA_25'],
    'VBFHtoaato4b_mA_30': ['VBFHtoaato4b_mA_30'],
    'VBFHtoaato4b_mA_35': ['VBFHtoaato4b_mA_35'],
    'VBFHtoaato4b_mA_40': ['VBFHtoaato4b_mA_40'],
    'VBFHtoaato4b_mA_45': ['VBFHtoaato4b_mA_45'],
    'VBFHtoaato4b_mA_50': ['VBFHtoaato4b_mA_50'],
    'VBFHtoaato4b_mA_55': ['VBFHtoaato4b_mA_55'],
    'VBFHtoaato4b_mA_60': ['VBFHtoaato4b_mA_60'],

    'WHtoaato4b_mA_12': ['WHtoaato4b_mA_12'],
    'WHtoaato4b_mA_15': ['WHtoaato4b_mA_15'],
    'WHtoaato4b_mA_20': ['WHtoaato4b_mA_20'],
    'WHtoaato4b_mA_25': ['WHtoaato4b_mA_25'],
    'WHtoaato4b_mA_30': ['WHtoaato4b_mA_30'],
    'WHtoaato4b_mA_35': ['WHtoaato4b_mA_35'],
    'WHtoaato4b_mA_40': ['WHtoaato4b_mA_40'],
    'WHtoaato4b_mA_45': ['WHtoaato4b_mA_45'],
    'WHtoaato4b_mA_50': ['WHtoaato4b_mA_50'],
    'WHtoaato4b_mA_55': ['WHtoaato4b_mA_55'],
    'WHtoaato4b_mA_60': ['WHtoaato4b_mA_60'],

    'ZHtoaato4b_mA_12': ['ZHtoaato4b_mA_12'],
    'ZHtoaato4b_mA_15': ['ZHtoaato4b_mA_15'],
    'ZHtoaato4b_mA_20': ['ZHtoaato4b_mA_20'],
    'ZHtoaato4b_mA_25': ['ZHtoaato4b_mA_25'],
    'ZHtoaato4b_mA_30': ['ZHtoaato4b_mA_30'],
    'ZHtoaato4b_mA_35': ['ZHtoaato4b_mA_35'],
    'ZHtoaato4b_mA_40': ['ZHtoaato4b_mA_40'],
    'ZHtoaato4b_mA_45': ['ZHtoaato4b_mA_45'],
    'ZHtoaato4b_mA_50': ['ZHtoaato4b_mA_50'],
    'ZHtoaato4b_mA_55': ['ZHtoaato4b_mA_55'],
    'ZHtoaato4b_mA_60': ['ZHtoaato4b_mA_60'],

    'ttHtoaato4b_mA_12': ['ttHtoaato4b_mA_12'],
    'ttHtoaato4b_mA_15': ['ttHtoaato4b_mA_15'],
    'ttHtoaato4b_mA_20': ['ttHtoaato4b_mA_20'],
    'ttHtoaato4b_mA_25': ['ttHtoaato4b_mA_25'],
    'ttHtoaato4b_mA_30': ['ttHtoaato4b_mA_30'],
    'ttHtoaato4b_mA_35': ['ttHtoaato4b_mA_35'],
    'ttHtoaato4b_mA_40': ['ttHtoaato4b_mA_40'],
    'ttHtoaato4b_mA_45': ['ttHtoaato4b_mA_45'],
    'ttHtoaato4b_mA_50': ['ttHtoaato4b_mA_50'],
    'ttHtoaato4b_mA_55': ['ttHtoaato4b_mA_55'],
    'ttHtoaato4b_mA_60': ['ttHtoaato4b_mA_60'],

    'QCD_bEnr': ['QCD_bEnr'],
    'QCD_BGen': ['QCD_BGen'],
    'QCD_Incl': ['QCD_Incl'],
    'TT0l': ['TT0l'],
    'TT1l': ['TT1l'],
    'TT2l': ['TT2l'],
    'STop_t': ['STop_t'],
    'STbar_t': ['STbar_t'],
    'ST_s_0l': ['ST_s_0l'],
    'ST_s_1l': ['ST_s_1l'],
    'STop_tW_Incl': ['STop_tW_Incl'],
    'STbar_tW_Incl': ['STbar_tW_Incl'],
    'STop_tW_12l': ['STop_tW_12l'],
    'STbar_tW_12l': ['STbar_tW_12l'],
    'ttZ': ['ttZ'],    
    'ttW': ['ttW'],
    'tZq': ['tZq'],
    'Zqq': ['Zqq'],
    'Zvv': ['Zvv'],
    
    'Zll': ['Zll'],
    'Wqq': ['Wqq'],
    'Wlv': ['Wlv'],
    
    'ZZ': ['ZZ'],
    'WZ': ['WZ'],
    'WW': ['WW'],
    
    'ZZZ': ['ZZZ'],
    'WZZ': ['WZZ'],
    'WWZ': ['WWZ'],
    'WWW': ['WWW'],
    'ggH': ['ggH'],

}
systematics_forData_dict = {'noweight': 'Nom'}
#systematics_dict = {'Nom': 'Nom'} #{'central': 'Nom'}
systematics_dict = {
    'Nom': 'Nom',
    'PUUp': 'PUUp',
    'PUDown': 'PUDown',   

}

histograms_list = [
    'hLeadingFatJetMass_vs_massA_Hto4b_avg',
    'hLeadingFatJetMSoftDrop_vs_massA_Hto4b_avg',
    'hLeadingFatJetParticleNet_massH_Hto4b_avg_vs_massA_Hto4b_avg',
]
histograms_dict = {
    'hLeadingFatJetMass_vs_massA_Hto4b_avg':                        'mass',
    'hLeadingFatJetMSoftDrop_vs_massA_Hto4b_avg':                   'msoft',
    'hLeadingFatJetParticleNet_massH_Hto4b_avg_vs_massA_Hto4b_avg': 'pnet',    
}
nRebinsX = 10
nRebinsY = 4


selectionTags_dict = {
    'WP40': {
        'Pass': 'SRWP40',
        'Fail': 'SBWP80to40'
    },
    'WP60': {
        'Pass': 'SRWP60',
        'Fail': 'SBWP95to60'
    }, 
    'WP80': {
        'Pass': 'SRWP80',
        'Fail': 'SBWP99to80'
    },        
}    

In [11]:
for CAT, CAT_original in CATAGORIES.items():
    sOpDir = '%s/%s' % (sOpDir0, CAT)
    if not os.path.exists(sOpDir):   os.makedirs(sOpDir)
    if 'Zvv' in CAT:
        processes_dict['Data'] = ['MET_Run2018A', 'MET_Run2018B', 'MET_Run2018C', 'MET_Run2018D']

    sHistoNames_NotRead   = []
    sProcessNames_NotRead = {}
    for processNameToUse, processNameList in processes_dict.items():
        PROC = processNameToUse
        YEAR = Era
        
        sOpFile = '%s/%s_%s_%s.root' % (sOpDir, CAT,PROC,YEAR)
        fOpFile = uproot.recreate(sOpFile)

        #for histo_name in histograms_list:
        for histo_name, histo_name_toSave in histograms_dict.items():
            for selectionWP, selectionRegions_dict in selectionTags_dict.items():
                for selectionRegion, selectionRegionNameOriginal0 in selectionRegions_dict.items():
                    selectionRegionNameOriginal = selectionRegionNameOriginal0
                    #if ('Zvv' in CAT) or ('gg0l' in CAT):
                    #    selectionRegionNameOriginal = '%s_%s' %(CAT, selectionRegionNameOriginal0)
                    selectionRegionNameOriginal = '%s_%s' %(CAT_original, selectionRegionNameOriginal0)
                        
                    for systematic, systematic_toSave in systematics_dict.items():
                    
                        hAdded = None
                        for processName in processNameList:
                            systematicNameToUse = systematic 
                            if 'Run' in processName or 'ata' in processNameToUse: # for Data
                                systematics_forData_original = list(systematics_forData_dict.keys())[0]
                                systematic_toSave            = systematics_forData_dict[systematics_forData_original]
                                systematicNameToUse          = systematics_forData_original
                            histo_name_toUse_full = 'evt/%s/%s_%s_%s' % (processName, histo_name, selectionRegionNameOriginal, systematicNameToUse)
                            #print(f"{histo_name_toUse_full = }")
                            try:
                                h = fIpFile[histo_name_toUse_full].to_hist()
                            except:
                                # histogram could not read
                                print(f"{histo_name_toUse_full = } could not read")
                                sHistoNames_NotRead.append(histo_name_toUse_full)
                                if processNameToUse not in sProcessNames_NotRead.keys():
                                    sProcessNames_NotRead[processNameToUse] = []
                                if processName not in sProcessNames_NotRead[processNameToUse]:
                                    sProcessNames_NotRead[processNameToUse].append( processName )

                                continue
                            h = h[::hist.rebin(nRebinsX), ::hist.rebin(nRebinsY)]
                            #print(f"After {h.axes = }, {h.axes[0] =  }")
                            if hAdded == None: hAdded = h
                            else:              hAdded = hAdded + h

                        if not hAdded: continue

                        ## Set bins with nEvents < 0 to nEvents = 0
                        nEvts          = hAdded.values()
                        varNEvts       = hAdded.variances()                
                        nEvts_modified = np.where(
                            (nEvts < 0),
                            np.full_like(nEvts, 1e-6),
                            nEvts
                        )
                        hAdded[:, :] = np.stack((nEvts_modified, varNEvts), axis=-1)
        
                        '''
                        #histoNameToSave = '%s/%s_%s_%s' % (processNameToUse, histo_name,selectionTagNameToUse,systematic)
                        histoNameToSave = '%s/%s_%s' % (processNameToUse, histo_name,selectionTagNameToUse)
                        print(f"{histoNameToSave = }")
                        fOpFile[histoNameToSave] = hAdded
                        '''

                        #histoNameToSave = '%s_%s' % (histo_name,selectionTagNameToUse)
                        MASS = histo_name_toSave
                        WP = selectionWP
                        PassOrFail = selectionRegion
                        SYST = systematic_toSave
                        histoNameToSave = '%s_%s_%s_%s_%s_%s_%s' % (CAT, PROC, YEAR, MASS, WP, PassOrFail, SYST)
                        fOpFile[histoNameToSave] = hAdded

        fOpFile.close()
    




In [12]:
print(f"sHistoNames_NotRead: ")
for sHistoName_NotRead in sHistoNames_NotRead:
    print(f"\t {sHistoName_NotRead}")

sHistoNames_NotRead: 


In [13]:
#print(f"{json.dumps(sProcessNames_NotRead, indent=4) = }")
print("sProcessNames_NotRead: ")
print(json.dumps(sProcessNames_NotRead, indent=4))

sProcessNames_NotRead: 
{}
